In [1]:
import asdf
import numpy as np

# Introduction

ASDF files store their information using a tree structure. This allows the stored information be be
hierarchically organized within the file. Without any extensions, this tree is a nested combination
of basic data structures: maps, lists, arrays, strings, and numbers. In Python these types correspond
to `dict`, `list`, `np.ndarray`, `str`, and `int`, `float`, `complex`. Where `np.ndarray` is treated in
a special way. These data types can be extended to other types and objects using extensions.

In the ASDF python library, this tree can be created using a Python dictionary using key/value pairs.
Indeed, one can interact with any `AsdfFile` tree as if it were a dictionary. Note that due to limits
imposed by Python, keys must be `bool`, `int`, or `str` types only, while data information can be any
of the above data types.

# Creating ASDF files using basic python types

Lets first create an ASDF file with the key/value pair `"hello": "world"`:

In [2]:
tree = {"hello": "world"}
ff = asdf.AsdfFile(tree)
ff.write_to("hello.asdf")
ff["hello"]

'world'

Open the `hello.asdf` file in your favorite text editor. You should see a something that looks like:
```
#ASDF 1.0.0
#ASDF_STANDARD 1.5.0
%YAML 1.1
%TAG ! tag:stsci.edu:asdf/
--- !core/asdf-1.1.0
asdf_library: !core/software-1.0.0 {author: The ASDF Developers, homepage: 'http://github.com/asdf-format/asdf',
  name: asdf, version: 2.12.0}
history:
  extensions:
  - !core/extension_metadata-1.0.0
    extension_class: asdf.extension.BuiltinExtension
    software: !core/software-1.0.0 {name: asdf, version: 2.12.0}
hello: world
...
```
Notice that the file contains more information than just the `"hello": "world"` key value that we
entered. It contains information on the library used to create the file under `asdf_library`, and
information on what the ASDF library needs (schemas, extensions, etc.) to deserialize the stored 
data under `history`. 

Next lets create a file that stores information using all the other basic python types
(avoiding arrays for now):

In [3]:
tree = {
    "hello": "world",
    "foo": 42,
    "bar": 3.14,
    "imaginary": complex(2, 3),
    "animals": ["cat", "dog", "bird"],
    "data": {"mean": 3.14, "std": 2.71},
}
ff = asdf.AsdfFile(tree)
ff.write_to("example.asdf")

Now open `example.asdf` in your text editor. You should see something like:
```
#ASDF 1.0.0
#ASDF_STANDARD 1.5.0
%YAML 1.1
%TAG ! tag:stsci.edu:asdf/
--- !core/asdf-1.1.0
asdf_library: !core/software-1.0.0 {author: The ASDF Developers, homepage: 'http://github.com/asdf-format/asdf',
  name: asdf, version: 2.12.0}
history:
  extensions:
  - !core/extension_metadata-1.0.0
    extension_class: asdf.extension.BuiltinExtension
    software: !core/software-1.0.0 {name: asdf, version: 2.12.0}
animals: [cat, dog, bird]
bar: 3.14
data: {mean: 3.14, std: 2.71}
foo: 42
hello: world
imaginary: !core/complex-1.0.0 (2+3j)
...
```
Again observe that all the data we added to our tree is contained within our asdf file. Notice in
particular the `imaginary` data now has a yaml tag denoting that the data is a complex number, this
tag will be used by ASDF to correctly deserialize this data as a `complex` type later.

## Opening your ASDF files

Opening ASDF files can be simply done with the `asdf.open` command much like the standard Python `open`
command, with the exception that one must explicitly use the `mode` keyword to specify the open method

For example:

In [4]:
with asdf.open("example.asdf") as af:
    print(f'{af["hello"]=}')
    print(f'{af["foo"]=}')
    print(f'{af["bar"]=}')
    print(f'{af["imaginary"]=}')
    print(f'{af["animals"]=}')
    print(f'{af["data"]=}')

af["hello"]='world'
af["foo"]=42
af["bar"]=3.14
af["imaginary"]=(2+3j)
af["animals"]=['cat', 'dog', 'bird']
af["data"]={'mean': 3.14, 'std': 2.71}


As one can see this does indeed result in opening the file.

One can also update the file:

In [5]:
with asdf.open("example.asdf", mode="rw") as af:
    af["new"] = "cool new stuff"
    af.update()

with asdf.open("example.asdf") as af:
    print(f'{af["new"]=}')

af["new"]='cool new stuff'


# Creating ASDF files with numpy arrays

Beyond the maps, lists, strings, and numbers built into Python, ASDF can save arrays, in particular
numpy arrays (`nd.array`). Indeed, much of ASDF is dedicated to efficiently saving arrays.

For example if suppose we want to save a random 8x8 numpy array:

In [6]:
tree = {"random_array": np.random.rand(8, 8)}
ff = asdf.AsdfFile(tree)
ff.write_to("random.asdf")

Now opening this file in your text editor will result in something like:
```
#ASDF 1.0.0
#ASDF_STANDARD 1.5.0
%YAML 1.1
%TAG ! tag:stsci.edu:asdf/
--- !core/asdf-1.1.0
asdf_library: !core/software-1.0.0 {author: The ASDF Developers, homepage: 'http://github.com/asdf-format/asdf',
  name: asdf, version: 2.12.0}
history:
  extensions:
  - !core/extension_metadata-1.0.0
    extension_class: asdf.extension.BuiltinExtension
    software: !core/software-1.0.0 {name: asdf, version: 2.12.0}
random_array: !core/ndarray-1.0.0
  source: 0
  datatype: float64
  byteorder: little
  shape: [8, 8]
...
�BLK 0                             �da?���C��`�!rm��?�?e�?�q���?orKu�?8�)%�F�?��	�w5�?D� ����?L�BN]�?�F����?���`c�|?_!���z�?
{j��u�?�h����?_����?3!C¡�?؀Đ1��?>�}����?��yn��?�K�q���?}}�A
-�? ����?V�8�f1�?H��~�h�?G����?`(���?x��ui�?�6�Zx�?�27	ѻ�?L�-P>�?��c����?$}����?�p�}=��?�tc�i��?vWE���?dܞ���?�
Au	��?)����?@�V
�I�?�4*�շ?n��N�?��
1���?��=B��?��GU<��?@GWx��?�z��4��?�TtǄ�?��5����?,�3"��?��i����?�t{�7�?�'~��?����G�?ģ�
t��?�@����?C�y"�?��G����?ଧ�N�?~G����?B��h�?��49���?��e���?�#�eG�?�v@�F�? ��a��?#ASDF BLOCK INDEX
%YAML 1.1
---
- 517
...
```
Observe that at the end of the file that there is apparently some binary data. This binary data contains the information
in the random array we wrote. Indeed, when ASDF writes arrays to the file it stores them as binary data in a block after
the YAML section of the file rather in the section itself. Note that `random_array` in the YAML section stores some
information about the nature of the array and includes the `source` key. This `source` value references which binary block 
(in this case block `0`) the data is stored in.

Indeed if we update the file with another array to the file we get a second block:

In [7]:
with asdf.open("random.asdf", mode="rw") as ff:
    ff.tree.update({"new_array": np.random.rand(10, 10)})
    ff.update()

Opening `random.asdf` in your text editor will give something like:
```
#ASDF 1.0.0
#ASDF_STANDARD 1.5.0
%YAML 1.1
%TAG ! tag:stsci.edu:asdf/
--- !core/asdf-1.1.0
asdf_library: !core/software-1.0.0 {author: The ASDF Developers, homepage: 'http://github.com/asdf-format/asdf',
  name: asdf, version: 2.12.0}
history:
  extensions:
  - !core/extension_metadata-1.0.0
    extension_class: asdf.extension.BuiltinExtension
    software: !core/software-1.0.0 {name: asdf, version: 2.12.0}
new_array: !core/ndarray-1.0.0
  source: 1
  datatype: float64
  byteorder: little
  shape: [10, 10]
random_array: !core/ndarray-1.0.0
  source: 0
  datatype: float64
  byteorder: little
  shape: [8, 8]
...
�BLK 0                             j㳡�oR�H*Fk�Xtb�q�?6����?R����?�W�
��?��C�;��?y"g�M�?�a?L6V�? cl�yl?�rb ;��?Xvd���?�D߽���?�0��?
MGb��?S|�eΊ�?������?��%5(a�?Р��Qʰ?0n<@Qƣ?���+�?r�+(���?�t��?4�LH���?�=S�P)�?J��qD��?��<O�?��3u
�?ÿ� �6�?�c�n��?ʫ�Ӧr�?�(S
���?�c@�`�?sE�
�?�T���?��-����?H��񐇸?6Y��Ț�?rSei<�?�N�%��?�B��o�?��?.���?��L
���?DL�+J�?�j폫��?M*n �?��?��P�?
����?h�Jh::�?lr�X���?�	��b�?z����?�^��Q�?�`N��m�?��2'�Y�?��L���?HL���P�?/�]��?��hW�?,�����?����Zŭ?䠶M���?'�����?������?�q�hr��?�Ww�b��?�BLK 0                             +}5�v�፮�GhB� ��F�?�<�r�{�?&}��?�$�qѤ�?Zř���?��L�[�?�����4�?���*���?�FG�o�?�� o,��?z��Q�?���b���? �#�~o?j�	�v��?��H��L�?����S��?~/��w�?���KA�?T���OR�?���!���?z�zV��?���*�Z�?����1\�? �RnM��?��B;���?�$����?��r���?
q��l�?H`�ԃ�?�[zs�?�9+`���?���$ E�?)�bY�R�?k��8Ԕ�?^��o^ �?�&����?�h����?�{��Er�?�m0uc�?V�v7-l�?x�r��p�?j��$�?��8���?}<ί�1�?��k�z@�?��_�D�?9J6���?9l��8�?��'�k�?�Dv:VX�?���M�?��cB���?�u�WZ�?.v8g��?��
|�$�?,c��?��T�l�?�,�ݛ`�?�e'L��?��! ��?t�L�es�?�Ȁ�54�?�������? ���e��?:j�%*��?���Ó�?�Ґ�NZ�?�U���?�����?$��)��?L��
Us�?"w��c<�?�7g���?��p���?���-
�?P����?�2��0)�?~
��?n1ʼ�l�?�I�Ù�?�������?r��%C"�?v�~8�?<b�Z�6�?��yM��?�;�a�_�?�2��P�?��T��?V��y��?�O|)�?4��::��?�0�ô�?d�@U��?<B��x�?^��%�?�����?	���n�?��Jan��?�::ͻ��?L�l��?#ASDF BLOCK INDEX
%YAML 1.1
---
- 618
- 1184
...
```
This now has a `new_array` key, which contains a second `source: 1` meaning there is a second binary block.

Now observe that ASDF is smart about storing arrays as binary data; meaning that, if arrays are shared between
entries in the tree, the same binary block is used. Indeed, this extends to sharing views on the data:

In [8]:
duplicated_array = np.random.rand(8, 8)
multi_view_array = np.random.rand(8, 8)
tree = {
    "duplicated_array_0": duplicated_array,
    "duplicated_array_1": duplicated_array,
    "multi_view_array": multi_view_array,
    "new_view": multi_view_array[2:4, 3:6],
}
with asdf.open("random.asdf", mode="rw") as ff:
    ff.tree.update(tree)
    ff.update()

Opening `random.asdf` in your text editor once again gives something like:
```
#ASDF 1.0.0
#ASDF_STANDARD 1.5.0
%YAML 1.1
%TAG ! tag:stsci.edu:asdf/
--- !core/asdf-1.1.0
asdf_library: !core/software-1.0.0 {author: The ASDF Developers, homepage: 'http://github.com/asdf-format/asdf',
  name: asdf, version: 2.12.0}
history:
  extensions:
  - !core/extension_metadata-1.0.0
    extension_class: asdf.extension.BuiltinExtension
    software: !core/software-1.0.0 {name: asdf, version: 2.12.0}
duplicated_array_0: &id001 !core/ndarray-1.0.0
  source: 3
  datatype: float64
  byteorder: little
  shape: [8, 8]
duplicated_array_1: *id001
multi_view_array: !core/ndarray-1.0.0
  source: 2
  datatype: float64
  byteorder: little
  shape: [8, 8]
new_array: !core/ndarray-1.0.0
  source: 0
  datatype: float64
  byteorder: little
  shape: [10, 10]
new_view: !core/ndarray-1.0.0
  source: 2
  datatype: float64
  byteorder: little
  shape: [2, 3]
  offset: 152
  strides: [64, 8]
random_array: !core/ndarray-1.0.0
  source: 1
  datatype: float64
  byteorder: little
  shape: [8, 8]
...
                                                                                                                                                                                           �BLK 0                             +}5�v�፮�GhB� ��F�?�<�r�{�?&}��?�$�qѤ�?Zř���?��L�[�?�����4�?���*���?�FG�o�?�� o,��?z��Q�?���b���? �#�~o?j�	�v��?��H��L�?����S��?~/��w�?���KA�?T���OR�?���!���?z�zV��?���*�Z�?����1\�? �RnM��?��B;���?�$����?��r���?
q��l�?H`�ԃ�?�[zs�?�9+`���?���$ E�?)�bY�R�?k��8Ԕ�?^��o^ �?�&����?�h����?�{��Er�?�m0uc�?V�v7-l�?x�r��p�?j��$�?��8���?}<ί�1�?��k�z@�?��_�D�?9J6���?9l��8�?��'�k�?�Dv:VX�?���M�?��cB���?�u�WZ�?.v8g��?��
|�$�?,c��?��T�l�?�,�ݛ`�?�e'L��?��! ��?t�L�es�?�Ȁ�54�?�������? ���e��?:j�%*��?���Ó�?�Ґ�NZ�?�U���?�����?$��)��?L��
Us�?"w��c<�?�7g���?��p���?���-
�?P����?�2��0)�?~
��?n1ʼ�l�?�I�Ù�?�������?r��%C"�?v�~8�?<b�Z�6�?��yM��?�;�a�_�?�2��P�?��T��?V��y��?�O|)�?4��::��?�0�ô�?d�@U��?<B��x�?^��%�?�����?	���n�?��Jan��?�::ͻ��?L�l��?�BLK 0                             j㳡�oR�H*Fk�Xtb�q�?6����?R����?�W�
��?��C�;��?y"g�M�?�a?L6V�? cl�yl?�rb ;��?Xvd���?�D߽���?�0��?
MGb��?S|�eΊ�?������?��%5(a�?Р��Qʰ?0n<@Qƣ?���+�?r�+(���?�t��?4�LH���?�=S�P)�?J��qD��?��<O�?��3u
�?ÿ� �6�?�c�n��?ʫ�Ӧr�?�(S
���?�c@�`�?sE�
�?�T���?��-����?H��񐇸?6Y��Ț�?rSei<�?�N�%��?�B��o�?��?.���?��L
���?DL�+J�?�j폫��?M*n �?��?��P�?
����?h�Jh::�?lr�X���?�	��b�?z����?�^��Q�?�`N��m�?��2'�Y�?��L���?HL���P�?/�]��?��hW�?,�����?����Zŭ?䠶M���?'�����?������?�q�hr��?�Ww�b��?�BLK 0                             ��F~eNAev7��
�Daט�ț�?F02��	�?H����?Td�Â�?`���?�v�H��?���U��?�F�����?��3�h�?�2�j��?�,3dC��?�(����?�WI{G]�?��.b	O�?\ZwE��?p�:�R�?'����?FUO-"��?\	����?TY�9�J�?x�����?����/�?�@q��?�q,	%�?�yZ2.�?�r��lW�?H�ñ��?�[J �?�m��k-�?��Դ}�?�I���? <;4��?�����?�ҟK�<�?6�u[�H�?��v�x��?�#�fA��?���)X��?0��7?��?�?=^�?~�?��?�0M`	��?
a�����?��� �a�?zn��_��?д73YO�?������? �P���?6$�7O�?��
m��?f��[���?�FZ���?T�˿�?�:K�?P��f��?h�
u�^�?�u�+��?��J�<0�?������?�9��8�?�',u�?h��>l��?W�&��?��Q!,��?�BLK 0              6              �Z������\���[�i���?ȫ K��?��e���?9]�s��?�Df�e�? �`d(�?����?��0?��? 4L7�?s�-CmR�?n���?�?	�'9q�?�w<ED�?�����?������?��$*��?�W�t���?�䑁�[�?pȴCr�?����(��?�������?-nŤ� �?�.�9�[�?���H�b�?�&ť��?�T�^�?X �!��?�\�����?�s6���?�xz&�k�?�᥈6��?y:����?�E�&�?��u��-�?���`��?P��I[�?�&\�
Þ?:����?@���l��?L�$�L��? �Ma�T�? �w�4��?x5��u�?�����O�?�W�B~��?>aG�?v��� �?��)M�?�����?�� ?��?U�4�u�?�}�"�?w<N����?�����?X�����?��9�\+�?���i��?�Ke?��?.�d4c�?'�iɜ��?'�w���?��#ڪ��?�z؜ڹ�?�� ��?                                                      #ASDF BLOCK INDEX
%YAML 1.1
---
- 1184
- 2038
- 2604
- 3170
...
```
First, note that `duplicated_array_1` simply listed using a yaml anchor which points to `duplicated_array_0`. Second,
observe that the `source` in both `multi_view_array` and `new_array` are the same value (`2`) rather than distinct values.
So ASDF did not unnecessary duplicate the binary data.

# Serializing other objects

The ASDF library supports writing extensions for objects outside of the ASDF-standard, which we will explain
in detail in another lecture. Assuming that one has installed an ASDF extension to support some custom Python
objects, ASDF will be able to seamlessly save those objects with no additional effort.

For example, as part of the install for this course we installed the `asdf-astropy` package, which provides
extensions for writing many `astropy` objects.

In [9]:
import astropy

quantity = 50 * astropy.units.m
model = astropy.modeling.models.Gaussian2D(quantity, 2, 3, 4, 5)
time = astropy.time.Time("J2000")
coord = astropy.coordinates.ICRS(ra=1 * astropy.units.deg, dec=2 * astropy.units.deg)

tree = {
    "quantity": quantity,
    "model": model,
    "time": time,
    "coord": coord,
}

ff = asdf.AsdfFile(tree)
ff.write_to("astropy.asdf")

Notice that since `asdf-astropy` was installed, no extra effort was required to write these objects
even though they clearly fall outside the ASDF-standard objects we previously discussed.

Moreover examining `astropy.asdf` in your text editor results in:
```
#ASDF 1.0.0
#ASDF_STANDARD 1.5.0
%YAML 1.1
%TAG ! tag:stsci.edu:asdf/
--- !core/asdf-1.1.0
asdf_library: !core/software-1.0.0 {author: The ASDF Developers, homepage: 'http://github.com/asdf-format/asdf',
  name: asdf, version: 2.12.0}
history:
  extensions:
  - !core/extension_metadata-1.0.0
    extension_class: asdf.extension.BuiltinExtension
    software: !core/software-1.0.0 {name: asdf, version: 2.12.0}
  - !core/extension_metadata-1.0.0
    extension_class: asdf.extension._manifest.ManifestExtension
    extension_uri: asdf://asdf-format.org/astronomy/coordinates/extensions/coordinates-1.0.0
    software: !core/software-1.0.0 {name: asdf-astropy, version: 0.2.1}
  - !core/extension_metadata-1.0.0
    extension_class: asdf.extension._manifest.ManifestExtension
    extension_uri: asdf://asdf-format.org/core/extensions/core-1.5.0
    software: !core/software-1.0.0 {name: asdf-astropy, version: 0.2.1}
  - !core/extension_metadata-1.0.0
    extension_class: asdf.extension._manifest.ManifestExtension
    extension_uri: asdf://asdf-format.org/transform/extensions/transform-1.5.0
    software: !core/software-1.0.0 {name: asdf-astropy, version: 0.2.1}
coord: !<tag:astropy.org:astropy/coordinates/frames/icrs-1.1.0>
  data: !<tag:astropy.org:astropy/coordinates/representation-1.0.0>
    components:
      lat: !<tag:astropy.org:astropy/coordinates/latitude-1.0.0> {unit: !unit/unit-1.0.0 deg,
        value: 2.0}
      lon: !<tag:astropy.org:astropy/coordinates/longitude-1.0.0>
        unit: !unit/unit-1.0.0 deg
        value: 1.0
        wrap_angle: !<tag:astropy.org:astropy/coordinates/angle-1.0.0> {unit: !unit/unit-1.0.0 deg,
          value: 360.0}
    type: UnitSphericalRepresentation
  frame_attributes: {}
model: !transform/gaussian2d-1.0.0
  amplitude: !unit/quantity-1.1.0 {unit: !unit/unit-1.0.0 m, value: 50.0}
  bounding_box:
  - [-24.5, 30.5]
  - [-20.0, 24.0]
  bounds:
    x_stddev: [1.1754943508222875e-38, null]
    y_stddev: [1.1754943508222875e-38, null]
  inputs: [x, y]
  outputs: [z]
  theta: 0.0
  x_mean: 2.0
  x_stddev: 4.0
  y_mean: 3.0
  y_stddev: 5.0
quantity: !unit/quantity-1.1.0 {unit: !unit/unit-1.0.0 m, value: 50.0}
time: !time/time-1.1.0 {scale: tt, value: J2000.000}
...
``` 
Which clearly shows that we were successful in saving all the `astropy` objects we intended to.